# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 3 2022 12:20PM,238774,19,8100611,"Emersa Waterbox, LLC",Released
1,Jun 3 2022 12:20PM,238774,19,8100612,"Emersa Waterbox, LLC",Released
2,Jun 3 2022 12:09PM,238772,10,Enova-10071,Emerginnova,Released
3,Jun 3 2022 12:09PM,238772,10,Enova-10072,Emerginnova,Released
4,Jun 3 2022 12:02PM,238771,12,8100105,LF of America Corp.,Released
5,Jun 3 2022 11:59AM,238765,20,7814329,"ACI Healthcare USA, Inc.",Released
6,Jun 3 2022 11:59AM,238765,20,7817625-BO,"ACI Healthcare USA, Inc.",Released
7,Jun 3 2022 11:59AM,238765,20,7798894-BO,"ACI Healthcare USA, Inc.",Released
8,Jun 3 2022 11:59AM,238765,20,7830799-BO,"ACI Healthcare USA, Inc.",Released
9,Jun 3 2022 11:59AM,238765,20,7798893-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,238769,Released,14
44,238770,Released,1
45,238771,Released,1
46,238772,Released,2
47,238774,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238769,NaN,NaN,14.0
238770,NaN,NaN,1.0
238771,NaN,NaN,1.0
238772,NaN,NaN,2.0
238774,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238670,3.0,8.0,1.0
238682,6.0,13.0,1.0
238711,0.0,0.0,1.0
238713,0.0,0.0,18.0
238718,1.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238670,3,8,1
238682,6,13,1
238711,0,0,1
238713,0,0,18
238718,1,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238670,3,8,1
1,238682,6,13,1
2,238711,0,0,1
3,238713,0,0,18
4,238718,1,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238670,3,8,1
1,238682,6,13,1
2,238711,,,1
3,238713,,,18
4,238718,1,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 3 2022 12:20PM,238774,19,"Emersa Waterbox, LLC"
2,Jun 3 2022 12:09PM,238772,10,Emerginnova
4,Jun 3 2022 12:02PM,238771,12,LF of America Corp.
5,Jun 3 2022 11:59AM,238765,20,"ACI Healthcare USA, Inc."
41,Jun 3 2022 11:58AM,238769,10,ISDIN Corporation
55,Jun 3 2022 11:58AM,238770,12,LF of America Corp.
56,Jun 3 2022 11:55AM,238767,12,LF of America Corp.
57,Jun 3 2022 11:55AM,238766,10,"Senseonics, Incorporated"
58,Jun 3 2022 11:49AM,238764,12,LF of America Corp.
59,Jun 3 2022 11:48AM,238763,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 3 2022 12:20PM,238774,19,"Emersa Waterbox, LLC",,,2
1,Jun 3 2022 12:09PM,238772,10,Emerginnova,,,2
2,Jun 3 2022 12:02PM,238771,12,LF of America Corp.,,,1
3,Jun 3 2022 11:59AM,238765,20,"ACI Healthcare USA, Inc.",,,36
4,Jun 3 2022 11:58AM,238769,10,ISDIN Corporation,,,14
5,Jun 3 2022 11:58AM,238770,12,LF of America Corp.,,,1
6,Jun 3 2022 11:55AM,238767,12,LF of America Corp.,,,1
7,Jun 3 2022 11:55AM,238766,10,"Senseonics, Incorporated",,,1
8,Jun 3 2022 11:49AM,238764,12,LF of America Corp.,,,1
9,Jun 3 2022 11:48AM,238763,22,"NBTY Global, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 12:20PM,238774,19,"Emersa Waterbox, LLC",2,,
1,Jun 3 2022 12:09PM,238772,10,Emerginnova,2,,
2,Jun 3 2022 12:02PM,238771,12,LF of America Corp.,1,,
3,Jun 3 2022 11:59AM,238765,20,"ACI Healthcare USA, Inc.",36,,
4,Jun 3 2022 11:58AM,238769,10,ISDIN Corporation,14,,
5,Jun 3 2022 11:58AM,238770,12,LF of America Corp.,1,,
6,Jun 3 2022 11:55AM,238767,12,LF of America Corp.,1,,
7,Jun 3 2022 11:55AM,238766,10,"Senseonics, Incorporated",1,,
8,Jun 3 2022 11:49AM,238764,12,LF of America Corp.,1,,
9,Jun 3 2022 11:48AM,238763,22,"NBTY Global, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 12:20PM,238774,19,"Emersa Waterbox, LLC",2,,
1,Jun 3 2022 12:09PM,238772,10,Emerginnova,2,,
2,Jun 3 2022 12:02PM,238771,12,LF of America Corp.,1,,
3,Jun 3 2022 11:59AM,238765,20,"ACI Healthcare USA, Inc.",36,,
4,Jun 3 2022 11:58AM,238769,10,ISDIN Corporation,14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 12:20PM,238774,19,"Emersa Waterbox, LLC",2.0,NaN,NaN
1,Jun 3 2022 12:09PM,238772,10,Emerginnova,2.0,NaN,NaN
2,Jun 3 2022 12:02PM,238771,12,LF of America Corp.,1.0,NaN,NaN
3,Jun 3 2022 11:59AM,238765,20,"ACI Healthcare USA, Inc.",36.0,NaN,NaN
4,Jun 3 2022 11:58AM,238769,10,ISDIN Corporation,14.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 12:20PM,238774,19,"Emersa Waterbox, LLC",2.0,0.0,0.0
1,Jun 3 2022 12:09PM,238772,10,Emerginnova,2.0,0.0,0.0
2,Jun 3 2022 12:02PM,238771,12,LF of America Corp.,1.0,0.0,0.0
3,Jun 3 2022 11:59AM,238765,20,"ACI Healthcare USA, Inc.",36.0,0.0,0.0
4,Jun 3 2022 11:58AM,238769,10,ISDIN Corporation,14.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4297338,62.0,26.0,4.0
12,1193801,5.0,8.0,0.0
15,1432519,26.0,7.0,0.0
16,477493,2.0,0.0,0.0
18,477441,2.0,0.0,0.0
19,477492,3.0,0.0,1.0
20,477447,37.0,13.0,6.0
22,238763,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4297338,62.0,26.0,4.0
1,12,1193801,5.0,8.0,0.0
2,15,1432519,26.0,7.0,0.0
3,16,477493,2.0,0.0,0.0
4,18,477441,2.0,0.0,0.0
5,19,477492,3.0,0.0,1.0
6,20,477447,37.0,13.0,6.0
7,22,238763,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,62.0,26.0,4.0
1,12,5.0,8.0,0.0
2,15,26.0,7.0,0.0
3,16,2.0,0.0,0.0
4,18,2.0,0.0,0.0
5,19,3.0,0.0,1.0
6,20,37.0,13.0,6.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,62.0
1,12,Released,5.0
2,15,Released,26.0
3,16,Released,2.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,22
Status,,,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0
Executing,26.0,8.0,7.0,0.0,0.0,0.0,13.0,1.0
Released,62.0,5.0,26.0,2.0,2.0,3.0,37.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,22
0,Completed,4.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0
1,Executing,26.0,8.0,7.0,0.0,0.0,0.0,13.0,1.0
2,Released,62.0,5.0,26.0,2.0,2.0,3.0,37.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,22
0,Completed,4.0,0.0,0.0,0.0,0.0,1.0,6.0,0.0
1,Executing,26.0,8.0,7.0,0.0,0.0,0.0,13.0,1.0
2,Released,62.0,5.0,26.0,2.0,2.0,3.0,37.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()